# CS295/395: Secure Distributed Computation
## Homework 4

## Definitions

In [ ]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import urllib.request
import galois
GF = galois.GF(2 ** 13 - 1)

class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self):
        """Initialize the field size and dictionary to hold received messages."""
        self.input = None
        self.output = None
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

# Generate Shamir shares for secret v with threshold t and number of shares n
def shamir_share(v, t, n):
    coefficients = GF([GF.Random() for _ in range(t-1)] + [v])
    poly = galois.Poly(coefficients)
    shares = [(GF(x), poly(GF(x))) for x in range(1, n+1)]
    return shares

# Reconstruct the secret from at least t Shamir shares
def reconstruct(shares):
    xs = GF([s[0] for s in shares])
    ys = GF([s[1] for s in shares])
    poly = galois.lagrange_poly(xs, ys)
    #print(poly)
    secret = poly(0)
    
    return secret

## Question 1 (20 points)

Describe a protocol to multiply three input numbers. The input numbers will be secret-shared according to a $(t,n)$ Shamir secret sharing scheme before the protocol starts, and each party will receive one share of each number. Each party should output *one share of the product*, using a $(t, n)$ Shamir secret sharing scheme (i.e. the threshold for the output should be the same as the threshold for the input).

\begin{equation*}
\textbf{Functionality: Multiply Three Numbers}\\
\fbox{$\mathcal{F}(a, b, c) = a \cdot b \cdot c$}
\end{equation*}

**HINT**: reference the in-class exercise from 9/19/2022.

YOUR ANSWER HERE

## Question 2

Implement your protocol from question 1.

In [ ]:
class MultThreeParty(Party):
    def round1(self, parties, a_shr, b_shr, c_shr, t):
        self.input = (a_shr, b_shr, c_shr)
        self.c_shr = c_shr # save this one for later
        self.parties = parties
        n = len(parties)
        assert t <= n/2
        
        # YOUR CODE HERE
        raise NotImplementedError()

    def round2(self):
        n = len(self.parties)
        
        # YOUR CODE HERE
        raise NotImplementedError()
    
    def round3(self):
        n = len(self.parties)
        
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# TEST CASE for question 2

NUM_PARTIES = 6
# (t, n)-Shamir scheme
n = NUM_PARTIES
t = 3

shares1 = shamir_share(5, t, n)
shares2 = shamir_share(6, t, n)
shares3 = shamir_share(7, t, n)

parties = [MultThreeParty() for _ in range(NUM_PARTIES)]

for p,s1,s2,s3 in zip(parties, shares1, shares2, shares3):
    p.round1(parties, s1, s2, s3, t)
for p in parties:
    p.round2()
for p in parties:
    p.round3()
for p in parties:
    # print(p.get_view())
    print(p.output)

output_shares = [p.output for p in parties]
print('Reconstruction, with all shares:', reconstruct(output_shares))
print('Reconstruction, with 3 shares:', reconstruct(output_shares[:3]))
print('Reconstruction, with 2 shares:', reconstruct(output_shares[:2]))

assert reconstruct(output_shares) == 210
assert reconstruct(output_shares[:3]) == 210
assert reconstruct(output_shares[:2]) != 210

## Question 3

Describe a protocol to compute the product of a list of $k$ numbers. The input numbers will be secret-shared according to a $(t,n)$ Shamir secret sharing scheme before the protocol starts, and each party will receive a list containing one share of each number. Each party should output *one share of the product*, using a $(t, n)$ Shamir secret sharing scheme (i.e. the threshold for the output should be the same as the threshold for the input).

\begin{equation*}
\textbf{Functionality: Product of $k$ Numbers}\\
\fbox{$\mathcal{F}(x_1, \dots, x_k) = \prod_{i=1}^k x_i$}
\end{equation*}

**HINT**: This problem is intentionally open-ended. The number of rounds of communication between the parties will depend on $k$.

YOUR ANSWER HERE

## Question 4

Implement your protocol from question 3.

In [ ]:
class MultListParty(Party):
    def round1(self, parties, xs, t):
        self.input = xs
        self.xs = xs
        self.parties = parties
        self.is_done = False
        n = len(parties)
        assert t <= n/2
        
        # YOUR CODE HERE
        raise NotImplementedError()

    def roundn(self, round):
        n = len(self.parties)
        
        # YOUR CODE HERE
        raise NotImplementedError()

In [ ]:
# Driver function for question 4
# NOTE: you can modify this function, if it helps for your implementation
# You may also want to uncomment pieces of the function for debugging
def run_list_prod():
    NUM_PARTIES = 6
    # (t, n)-Shamir scheme
    n = NUM_PARTIES
    t = 3
    k = 5

    parties = [MultListParty() for _ in range(NUM_PARTIES)]
    nums = [i+1 for i in range(k)]
    shares = [shamir_share(x, t, n) for x in nums]
    share_groups = list(zip(*shares))

    for p,xs in zip(parties, share_groups):
        p.round1(parties, xs, t)

    round_num = 2
    while not parties[0].is_done:
        for p in parties:
            p.roundn(round_num)       
        round_num += 1

    output_shares = [p.output for p in parties]
    
    print('Reconstruction, with all shares:', reconstruct(output_shares))
    print('Reconstruction, with 3 shares:', reconstruct(output_shares[:3]))
    print('Reconstruction, with 2 shares:', reconstruct(output_shares[:2]))
    return nums, output_shares

In [ ]:
# TEST CASE for question 4
nums, output_shares = run_list_prod()
assert reconstruct(output_shares) == np.prod(nums)
assert reconstruct(output_shares[:3]) == np.prod(nums)
assert reconstruct(output_shares[:2]) != np.prod(nums)